In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
#数据载入---有其他方法
mnist = input_data.read_data_sets("MNIST_data",one_hot=True) #注意输出标签是one_hot类型

#按批次进行计算
#设置每个批次大小
batch_size = 100
#获取批次数量，注意：我们训练时使用的是mnist.train训练集数据
n_batch = mnist.train.num_examples // batch_size #注意：//也是除法，返回整数

#开始定义占位符号
x = tf.placeholder(tf.float32,[None,784]) #由于每张图片都是28×28像素，所以每个样本的特征都是784
y = tf.placeholder(tf.float32,[None,10]) #是分10类
keep_prob = tf.placeholder(tf.float32) #设置全局dropout参数,即设置每个神经元正常工作的概率
lr = tf.Variable(0.001,dtype=tf.float32) #学习率设置

#开始创建一个神经网络隐藏层L1含有10个激活单元
Weight_L1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1)) #产生截断正态分布随机数，取值范围为[ mean - 2 * stddev, mean + 2 * stddev ]  截断的产生正态分布的随机数，即随机数与均值的差值若大于两倍的标准差，则重新生成
biases_L1 = tf.Variable(tf.zeros([1,500])+0.1) #对于偏执单元的权重，我们没有必要进行随机初始化,但是可以设置初始值
Wx_plus_b_L1 = tf.matmul(x,Weight_L1)+biases_L1
L1 = tf.nn.tanh(Wx_plus_b_L1)
L1_drop = tf.nn.dropout(L1,keep_prob) #设置该L1层的dropout信息

Weight_L2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1)) #产生截断正态分布随机数，取值范围为[ mean - 2 * stddev, mean + 2 * stddev ]  截断的产生正态分布的随机数，即随机数与均值的差值若大于两倍的标准差，则重新生成
biases_L2 = tf.Variable(tf.zeros([1,300])+0.1) #对于偏执单元的权重，我们没有必要进行随机初始化,但是可以设置初始值
Wx_plus_b_L2 = tf.matmul(L1_drop,Weight_L2)+biases_L2
L2 = tf.nn.tanh(Wx_plus_b_L2)
L2_drop = tf.nn.dropout(L2,keep_prob) #设置该L1层的dropout信息

Weight_L3 = tf.Variable(tf.truncated_normal([300,200],stddev=0.1)) #产生截断正态分布随机数，取值范围为[ mean - 2 * stddev, mean + 2 * stddev ]  截断的产生正态分布的随机数，即随机数与均值的差值若大于两倍的标准差，则重新生成
biases_L3 = tf.Variable(tf.zeros([1,200])+0.1) #对于偏执单元的权重，我们没有必要进行随机初始化,但是可以设置初始值
Wx_plus_b_L3 = tf.matmul(L2_drop,Weight_L3)+biases_L3
L3 = tf.nn.tanh(Wx_plus_b_L3)
L3_drop = tf.nn.dropout(L3,keep_prob) #设置该L1层的dropout信息

Weight_L4 = tf.Variable(tf.truncated_normal([200,10],stddev=0.1)) #产生截断正态分布随机数，取值范围为[ mean - 2 * stddev, mean + 2 * stddev ]  截断的产生正态分布的随机数，即随机数与均值的差值若大于两倍的标准差，则重新生成
biases_L4 = tf.Variable(tf.zeros([1,10])+0.1) #对于偏执单元的权重，我们没有必要进行随机初始化,但是可以设置初始值
Wx_plus_b_L4 = tf.matmul(L3_drop,Weight_L4)+biases_L4
y_pred = tf.nn.softmax(Wx_plus_b_L4)

#定义代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred)) #这里我们使用交叉熵进行处理，使得加快训练速度
#使用梯度下降法
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

init = tf.global_variables_initializer()

#获取结果存放在bool列表中
correct_prediction = tf.equal(tf.arg_max(y,1),tf.arg_max(y_pred,1)) #注意：arg_max第二个参数是指定坐标信息，因为结果按行存储，所以1
#根据上面的bool列表，获取准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #tf.cast格式转换后，求解平均值


#开启会话
with tf.Session() as sess:
    sess.run(init)
    for iter_cnt in range(51): #整体数据迭代
        sess.run(tf.assign(lr,0.001*(0.95**iter_cnt))) #随着迭代次数，降低学习率，防止发散
        for batch_cnt in range(n_batch): #小批次数据分批次迭代
            batch_xs,batch_ys = mnist.train.next_batch(batch_size) #依照batch_size获取下一批次数据
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0}) #传入数据，进行训练。对于隐藏层每一层随机使用70%的激活单元正常工作，另外30%不工作防止过拟合
            
        #获取每一次整体数据迭代后的准确率
        #在我们进行训练权值时，我们需要设置dropout参数去降低拟合程度，我们训练完毕以后，需要我们预测时，是不需要进行甚至dropout参数的，所以下面我们都是设置为1
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0}) #注意：预测时，我们也要传入数据，并且传入的是测试集数据
        learn_rate = sess.run(lr)
        print("Iter: %d, Testing accuracy:%f----learn rate:%f"%(iter_cnt,test_acc,learn_rate))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0, Testing accuracy:0.952500----learn rate:0.001000
Iter: 1, Testing accuracy:0.959800----learn rate:0.000950
Iter: 2, Testing accuracy:0.967800----learn rate:0.000903
Iter: 3, Testing accuracy:0.972100----learn rate:0.000857
Iter: 4, Testing accuracy:0.972000----learn rate:0.000815
Iter: 5, Testing accuracy:0.972900----learn rate:0.000774
Iter: 6, Testing accuracy:0.976000----learn rate:0.000735
Iter: 7, Testing accuracy:0.976200----learn rate:0.000698
Iter: 8, Testing accuracy:0.973800----learn rate:0.000663
Iter: 9, Testing accuracy:0.976100----learn rate:0.000630
Iter: 10, Testing accuracy:0.977100----learn rate:0.000599
Iter: 11, Testing accuracy:0.976900----learn rate:0.000569
Iter: 12, Testing accuracy:0.978900----learn rate:0.000540
Iter: 13, Testing accuracy:0.980300----learn rate

In [ ]:
Iter: 48, Testing accuracy:0.981500----learn rate:0.000085
Iter: 49, Testing accuracy:0.981000----learn rate:0.000081
Iter: 50, Testing accuracy:0.981700----learn rate:0.000077